## Beta-2-adrenergic receptor (B2AR) membrane protein simulation with CHARMM
* https://openmm.github.io/openmm-cookbook/latest/notebooks/tutorials/Beta_2_adrenergic_receptor_B2AR_membrane_protein_simulation_with_CHARMM.html

In [4]:
# download and extract the files
# you need to de-comment the lines to run the code for the first time
#!wget https://openmm.org/tutorials_/b2ar_membrane/files/charmm-gui.zip -O ./data/charmm-gui.zip
#!tar xvf ./data/charmm-gui.zip -C ./data

In [9]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import nglview as nv
from sys import stdout

In [7]:
# Load the CHARMM files
psf = CharmmPsfFile('./data/charmm-gui/openmm/step5_charmm2omm.psf')
pdb = PDBFile('./data/charmm-gui/openmm/step5_charmm2omm.pdb')
params = CharmmParameterSet('./data/charmm-gui/toppar/par_all36_prot.prm',
                            './data/charmm-gui/toppar/top_all36_prot.rtf',
                            './data/charmm-gui/toppar/par_all36_lipid.prm',
                            './data/charmm-gui/toppar/top_all36_lipid.rtf',
                            './data/charmm-gui/toppar/toppar_water_ions.str')

In [8]:

# Create an openmm system by calling createSystem on psf
system = psf.createSystem(params, nonbondedMethod=CutoffNonPeriodic,
         nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin,   # Temperature of head bath
                                1/picosecond, # Friction coefficient
                                0.002*picoseconds) # Time step


In [18]:
view = nv.show_structure_file('./data/charmm-gui/openmm/step5_charmm2omm.pdb')
#view.add_representation('licorice', selection='water')
view.add_representation('cartoon', selection='protein')
view


NGLWidget()

In [19]:
simulation = Simulation(psf.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
print("Minimizing...")
simulation.minimizeEnergy()

Minimizing...


In [20]:

# Set up the reporters to report energies every 1000 steps.
simulation.reporters.append(PDBReporter('./results/13_output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
                            potentialEnergy=True, temperature=True))
# run simulation
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-356653.6162109375,257.0097890683596
2000,-336403.07763671875,291.13759641019266
3000,-332426.0322265625,298.47856224203804
4000,-333022.54638671875,298.93678539638444
5000,-332074.1806640625,300.5804396380163
6000,-332668.54296875,301.3076194891366
7000,-332559.32763671875,298.8697824502224
8000,-334628.94873046875,300.6511455554233
9000,-334128.396484375,299.40794662224715
10000,-336227.587890625,298.78323944917645


In [23]:
view = nv.show_structure_file('./results/13_output.pdb')
#nv.show_mdtrajectories(view, './results/13_output.pdb')
view.add_representation('cartoon', selection='protein')
view

NGLWidget()